In [1]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import warnings
import tika
from tika import parser
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import spacy
import re
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')
import multiprocessing
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
base = ['data/AS_', 'data/Ml_', 'data/Bd_', 'data/Hd_']
dados = []
num_data = []
for elem in range(0, len(base)):
    count = 0
    if elem != 0:
        final = 10
    else:
        final = 34
        
    for i in range(1, final):
        terminal = base[elem] + str(i) + '.pdf'
        dados.append(terminal)
        count+=1
    
    num_data.append(count)



In [3]:
print(dados)

['data/AS_1.pdf', 'data/AS_2.pdf', 'data/AS_3.pdf', 'data/AS_4.pdf', 'data/AS_5.pdf', 'data/AS_6.pdf', 'data/AS_7.pdf', 'data/AS_8.pdf', 'data/AS_9.pdf', 'data/AS_10.pdf', 'data/AS_11.pdf', 'data/AS_12.pdf', 'data/AS_13.pdf', 'data/AS_14.pdf', 'data/AS_15.pdf', 'data/AS_16.pdf', 'data/AS_17.pdf', 'data/AS_18.pdf', 'data/AS_19.pdf', 'data/AS_20.pdf', 'data/AS_21.pdf', 'data/AS_22.pdf', 'data/AS_23.pdf', 'data/AS_24.pdf', 'data/AS_25.pdf', 'data/AS_26.pdf', 'data/AS_27.pdf', 'data/AS_28.pdf', 'data/AS_29.pdf', 'data/AS_30.pdf', 'data/AS_31.pdf', 'data/AS_32.pdf', 'data/AS_33.pdf', 'data/Ml_1.pdf', 'data/Ml_2.pdf', 'data/Ml_3.pdf', 'data/Ml_4.pdf', 'data/Ml_5.pdf', 'data/Ml_6.pdf', 'data/Ml_7.pdf', 'data/Ml_8.pdf', 'data/Ml_9.pdf', 'data/Bd_1.pdf', 'data/Bd_2.pdf', 'data/Bd_3.pdf', 'data/Bd_4.pdf', 'data/Bd_5.pdf', 'data/Bd_6.pdf', 'data/Bd_7.pdf', 'data/Bd_8.pdf', 'data/Bd_9.pdf', 'data/Hd_1.pdf', 'data/Hd_2.pdf', 'data/Hd_3.pdf', 'data/Hd_4.pdf', 'data/Hd_5.pdf', 'data/Hd_6.pdf', 'data/

In [4]:
# data = ['data/Ml_1.pdf', 'data/Ml_2.pdf', 'data/Ml_3.pdf', 'data/Ml_4.pdf', 'data/Ml_5.pdf', 'data/Ml_6.pdf', 'data/Ml_7.pdf']
biblioteca = []
tika.initVM()
for i in range(0, len(dados)):
    
    parsed = parser.from_file(dados[i])
    metadados = parsed["metadata"]
    texto = parsed["content"]
    biblioteca.append(texto)

In [5]:
print(len(biblioteca))
# print(len(data))

60


In [6]:
# f = open ("artigo2.txt", "w")
# f.write (str(texto))
# f.close ()

In [7]:

# f = open ("artigo2.txt", "r")
# texto = f.read()

In [8]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [9]:
def split_pdf(string): 
    li = list(string.split(" ")) 
    return li

In [10]:
def first_clear(data):
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

    #remove links and mark
    data = [re.sub('(https:\S+)|(@)|(¿)', '', sent) for sent in data]
    
    data = [re.sub('(http:\S+)|(@)|(¿)', '', sent) for sent in data]
    
    data = [re.sub('(www.:\S+)|(@)|(¿)', '', sent) for sent in data]

    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]

    # remove spaces in begining and end
    data = [re.sub(' +', ' ', sent) for sent in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]

    return data

In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [12]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [13]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
def convert(s): 
    a = 0
    # initialization of string to "" 
    new = "" 
  
    # traverse in the string  
    for x in s: 
        new += x + ' '
        a+=1
  
    # return string  
    return new, a

In [15]:
data_cleared = []

In [16]:
total_char = 0
for i in range(0, len(biblioteca)):
    texto = biblioteca[i]
    data = split_pdf(texto)
    data = first_clear(data)
    d, a = convert(data)
    
    total_char += a
    data_cleared.append(d)


In [17]:
print(len(data_cleared))
print(total_char)

60
546022


In [18]:
df = pd.DataFrame(data_cleared)
df = df.rename(columns={0: "text"})

In [19]:
df.head()

,text
0,Accurate Methods for the Statistics of Surpri...
1,11/17/2019 A Companion to Digital Literary St...
2,fqi063 311..326 A Tool for Literary Studies: ...
3,A survey of modern authorship attribution met...
4,Linguistic Society of America is collaborat...


In [20]:
l = []
for i in range(0, len(dados)):
    if 'AS' in str(dados[i]):
        l.append('Analyze_Statistics')
    elif 'Ml' in str(dados[i]):
        l.append('Machine_Learning')
    elif 'Bd' in str(dados[i]):
        l.append('Big_Data')
    else:
        l.append('Digital_Humanities')

In [21]:
df['classe'] = l

In [22]:
df.head()

,text,classe
0,Accurate Methods for the Statistics of Surpri...,Analyze_Statistics
1,11/17/2019 A Companion to Digital Literary St...,Analyze_Statistics
2,fqi063 311..326 A Tool for Literary Studies: ...,Analyze_Statistics
3,A survey of modern authorship attribution met...,Analyze_Statistics
4,Linguistic Society of America is collaborat...,Analyze_Statistics


In [23]:
train, test = train_test_split(df, test_size=0.2, random_state=42)


In [24]:
import nltk
from nltk.corpus import stopwords
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

In [25]:
train_tagged = train.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.classe]), axis=1)
test_tagged = test.apply(
    lambda r: TaggedDocument(words=tokenize_text(r['text']), tags=[r.classe]), axis=1)

In [26]:
train_tagged


31    ([fqm012, 345..362, delta, for, middle, dutch—...
3     ([survey, of, modern, authorship, attribution,...
52    ([9-25_1, golumbia, 156-76, .indd, volume, 25,...
17    ([corpus, linguistics, and, the, study, of, li...
8     ([corpus, 2003, corpus, 2003, la, distance, in...
6     ([journal, of, machine, learning, research, 20...
40    ([large-scale, classification, of, fine-art, p...
4     ([linguistic, society, of, america, is, collab...
43    ([microsoft, word, forman-jmlr-fxselectionstud...
19    ([bayesian, analysis, of, multinomial, sequenc...
34    ([fqs003, 183..196, detecting, authorship, dec...
58    ([11/8/12, dhq, digital, humanities, quarterly...
25    ([digital, humanities, 2010, does, size, matte...
56    ([signs, symbols, and, discourses, new, direct...
15    ([11/17/2019, clustering, with, compression, f...
27    ([11/17/2019, from, babel, to, knowledge, data...
9     ([authorial, attribution, and, computational, ...
30    ([dating, medieval, english, charters, the

In [27]:
import multiprocessing

cores = multiprocessing.cpu_count()

In [28]:
model_dbow = Doc2Vec(dm=1, vector_size=500, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 48/48 [00:00<00:00, 97921.49it/s]


In [29]:
%%time
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 48/48 [00:00<00:00, 174459.79it/s]


CPU times: user 39.9 s, sys: 333 ms, total: 40.2 s
Wall time: 11.8 s


In [30]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [31]:

y_train, X_train = vec_for_learning(model_dbow, train_tagged)
y_test, X_test = vec_for_learning(model_dbow, test_tagged)

In [32]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [33]:
from sklearn.metrics import accuracy_score, f1_score

print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.5
Testing F1 score: 0.42735042735042733
